<a href="https://colab.research.google.com/github/nalet/bme.Advanced-Topics-in-Machine-Learning/blob/master/Assignments/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name: Nalet Meinen¶

# Assignment 2 ATML 2020
## Classification with limited data
ImageNet is a well known dataset with 1000 image classes. We will be working on a subset of the dataset (60k images, 100 classes, 600 images per class 80$\times$80 pixels, RGB) and train a model to classify an image into one of the 100 classes. The dataset is located under the "data" directory. Training and validation data splits are under "data/train" and "data/val" directories respectively. Both splits consist of 100 directories, each representing an object category.

In [1]:
import os.path
if not os.path.isfile('data.zip'):
  from google.colab import drive
  drive.mount('/content/drive')
  !cp -r "/content/drive/My Drive/ATML/Assignments/Assignment2/." .
  !unzip -qq data.zip
!pip install tqdm
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Mon Apr  6 15:59:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================

## Task 1. Implement ImageNetLimited class for data loading in datasets.py file

In [2]:
import numpy as np
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from datasets import ImageNetLimited
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print("This nootebook ran on",device.type,"With number of GPU:",torch.cuda.device_count())

train_dir = 'data/train'
validation_dir = 'data/val'
len_classes = 100

# write your code
_transforms = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(64,padding=0),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

_train_dataset = ImageFolder(train_dir, transform=train_transformations)
_val_dataset = ImageFolder(validation_dir, transform=_transforms)

train_dataset = ImageNetLimited(_train_dataset)
val_dataset = ImageNetLimited(_val_dataset)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8) #set num_workers = 0, to fix BrokenPipeError in Windows
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

This nootebook ran on cuda With number of GPU: 1


## Task 2. CNN Architecture
Design and implement a Convolutional Neural Network architecture for image classification in a **ConvNet** class in the notebook. Some examples of popular classification models are: AlexNet, VGG, ResNet, ... Justify your design choices in the report. The input to your model must be an image of size $64 \times 64$ pixels.

In [0]:
import torch.nn as nn

class Unit(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(Unit,self).__init__()
        

        self.conv = nn.Conv2d(in_channels=in_channels,kernel_size=3,out_channels=out_channels,stride=1,padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()

    def forward(self,input):
        output = self.conv(input)
        output = self.bn(output)
        output = self.relu(output)

        return output

class ConvNet(nn.Module):
    def __init__(self,num_classes=100):
        super(ConvNet,self).__init__()

        #Create 14 layers of the unit with max pooling in between
        self.unit1 = Unit(in_channels=3,out_channels=32)
        self.unit2 = Unit(in_channels=32, out_channels=32)
        self.unit3 = Unit(in_channels=32, out_channels=32)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.unit4 = Unit(in_channels=32, out_channels=64)
        self.unit5 = Unit(in_channels=64, out_channels=64)
        self.unit6 = Unit(in_channels=64, out_channels=64)
        self.unit7 = Unit(in_channels=64, out_channels=64)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.unit8 = Unit(in_channels=64, out_channels=128)
        self.unit9 = Unit(in_channels=128, out_channels=128)
        self.unit10 = Unit(in_channels=128, out_channels=128)
        self.unit11 = Unit(in_channels=128, out_channels=128)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.unit12 = Unit(in_channels=128, out_channels=128)
        self.unit13 = Unit(in_channels=128, out_channels=128)
        self.unit14 = Unit(in_channels=128, out_channels=128)

        self.avgpool = nn.AvgPool2d(kernel_size=4)
        
        #Add all the units into the Sequential layer in exact order
        self.net = nn.Sequential(self.unit1, self.unit2, self.unit3, self.pool1, self.unit4, self.unit5, self.unit6
                                 ,self.unit7, self.pool2, self.unit8, self.unit9, self.unit10, self.unit11, self.pool3,
                                 self.unit12, self.unit13, self.unit14, self.avgpool)

        self.fc = nn.Linear(128 * 4, num_classes)

    def forward(self, input):
        output = self.net(input)
        output = output.view(output.size(0), -1)
        output = self.fc(output)
        return output

## Task 3. Train Model
Implement training and evaluation code for your model. Choose an appropriate loss function and evaluate the model on the validation set using classification accuracy. You are not allowed to use a pre-trained model (must train from scratch on the provided data).<br>
<font color='red'>Your model should achieve an accuracy of at least 40.0% on the validation set (Model with performance smaller than 40.0% will result in 0 points for this task).</font><br>

In [4]:
from utils_train import fit
model_conv = ConvNet()
model_conv = model_conv.to(device)
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                                momentum=0.8,
                                weight_decay=1e-4)
n_epochs = 30
loss_fn = nn.CrossEntropyLoss()
curves_conv2 = fit(train_dataloader, val_dataloader, model_conv, optimizer, loss_fn, n_epochs)


Test average loss: 4.1257, accuracy: 5.805
Epoch 1/30: train_loss: 4.3257, train_accuracy: 3.7617, val_loss: 4.1257, val_accuracy: 5.8047



Test average loss: 3.8999, accuracy: 9.074
Epoch 2/30: train_loss: 3.9456, train_accuracy: 8.0417, val_loss: 3.8999, val_accuracy: 9.0739



Test average loss: 3.5756, accuracy: 13.998
Epoch 3/30: train_loss: 3.6203, train_accuracy: 12.9183, val_loss: 3.5756, val_accuracy: 13.9977



Test average loss: 3.5247, accuracy: 15.468
Epoch 4/30: train_loss: 3.3455, train_accuracy: 17.4533, val_loss: 3.5247, val_accuracy: 15.4681



Test average loss: 3.2590, accuracy: 19.762
Epoch 5/30: train_loss: 3.1191, train_accuracy: 21.7467, val_loss: 3.2590, val_accuracy: 19.7622


...


Test average loss: 4.8672, accuracy: 27.399
Epoch 26/30: train_loss: 0.5298, train_accuracy: 83.4383, val_loss: 4.8672, val_accuracy: 27.3991



Test average loss: 4.2622, accuracy: 29.194
Epoch 27/30: train_loss: 0.4720, train_accuracy: 85.1383, val_loss: 4.2622, val_accuracy: 29.1944



Test average loss: 4.6909, accuracy: 27.667
Epoch 28/30: train_loss: 0.4260, train_accuracy: 86.5133, val_loss: 4.6909, val_accuracy: 27.6671



Test average loss: 4.9140, accuracy: 27.945
Epoch 29/30: train_loss: 0.3935, train_accuracy: 87.4833, val_loss: 4.9140, val_accuracy: 27.9451


KeyboardInterrupt: ignored

## Task 4. Ablations
Try to find the best performing model by tuning the model design and hyper-parameters on the validation set. Perform ablation experiments to illustrate the effect of the most important hyper-parameters. Some examples of ablations: training parameters (e.g., optimizer, learning rates, batch size), network architecture (e.g., number of layers, number of units, activation function, normalization layers), model regularization (e.g., data augmentation, dropout, weight decay, early stopping), test-time augmentation, etc...  <br>**Perform at least 5 ablations and report the performance of each on the validation set.**

In [0]:
# write your code

## Task 5. Model Errors
Evaluate the trained model on the validation set and plot 10 random mistakes that your model made.

In [0]:
# write your code

## Task 6. Competition time!
Read the images from "data/test" folder. There are no labels for these images. Run your best model on these images and save the image IDs (names) and predicted label in a file LastName.csv. You will receive a link via email to upload the CSV file to  an online system which will give you the score of your model on the held-out test set. Top 5 students with at least 40% classification accuracy will obtain bonus points.

In [0]:
# write your code